## Compilation



In [1]:
cd $HOME/models/lif_cpp
# make clean
make

| lto-wrapper: | warning: | using      | serial | compilation | of   | 2 | LTRANS | jobs |
| [            | Babel    | evaluation | exited | with        | code | 0 | ]      |      |

## Notebook Configuration



### Settings



    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
    %run /home/leon/models/lif_cpp/notebooks/setup.py
    %matplotlib inline
    %config InlineBackend.figure_format = 'png'

    The autoreload extension is already loaded. To reload it, use:
      %reload_ext autoreload
    Python exe
    /home/leon/mambaforge/envs/dual_data/bin/python



### Imports



    import sys
    sys.path.insert(0, '/home/leon/models/lif_cpp')  
    REPO_PATH = "/home/leon/models/lif_cpp"
    
    import warnings
    # Ignore FutureWarning
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    
    import subprocess
    import pandas as pd
    from time import sleep
    from yaml import safe_load
    from scipy.stats import circmean
    from joblib import Parallel, delayed
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    from run_model import run_cpp, update_conf, restore_conf
    from analysis.decode import decode_bump, circcvl
    
    pal = [sns.color_palette("tab10")[0], sns.color_palette("tab10")[1]]



## Helpers



### simulations



    def run_ini_phi(repo_path, conf_name, name, ini_list, phi_list, X_str=None, X_gain=1, X_axis=0, gain=1):
    
      conf_path = repo_path + '/conf/' + conf_name
      config = safe_load(open(conf_path + ".yml", "r"))
      data_path = repo_path + '/data/simul/'
    
      restore_conf(conf_path + '.yml',
                   conf_path +'.yml.bak')
    
      update_conf(conf_path, 'Iext', float(config['Iext'][0] * gain), axis=0)
      update_X(conf_path, config, X_str=X_str, axis=X_axis, gain=X_gain)
    
      for ini in ini_list:
        for phi in phi_list:
          session = name + "_phi_%d_ini_%d" % (phi, ini)
          update_conf(conf_path, 'DATA_PATH', data_path + session)
          update_conf(conf_path, 'PHI_STIM', float(phi), axis=0)
    
          sleep(.2)
          run_cpp(session, conf_path = conf_path + ".yml")
          sleep(.2)
    
          subprocess.run([repo_path + '/src/mem_usage.sh'])
          subprocess.run([repo_path + '/src/cpu_usage.sh'])
    
      restore_conf(conf_path + '.yml.bak',
                   conf_path +'.yml')

    def update_X(conf_path, config, X_str=None, axis=0, gain=1):
    
      if X_str is not None:
        # if len(axis)==1:
        #   if axis is not None:
        #     if axis<0:
        #       for axis in range(np.array(config[X_str]).shape[0]):
        #         update_conf(conf_path, X_str, float(config[X_str][axis] * gain), axis=axis)
        #     else:
        #       update_conf(conf_path, X_str, float(config[X_str][axis] * gain), axis=axis)
        #   else:
        #     update_conf(conf_path, X_str, float(config[X_str] * gain), axis=None)
        # else:
        for ax in axis:
          update_conf(conf_path, X_str, float(config[X_str][ax] * gain), axis=ax)



### data



    def plot_smooth(data, ax, color):
        mean = data.mean(axis=0)  
        ci = smooth.std(axis=0, ddof=1) * 1.96
    
        # Plot
        ax.plot(mean, color=color)
        ax.fill_between(range(data.shape[1]), mean - ci, mean + ci, alpha=0.25, color=color)

    def get_data(session='/'):
      # Open the file in binary mode and read the data
      with open('/home/leon/models/lif_cpp/data/simul/'+ session + '/rates.txt', 'rb') as f:
        data = np.fromfile(f, dtype=np.float32)
      # Now 'data' is a numpy array containing your data
      return data.reshape(-1, 40000).T

    def get_rate(name, ini, phi):
         session = "/%s_phi_%d_ini_%d/" % (name, phi, ini)
         rate = get_data(session)
         return rate[:32000]
    
    def get_rates_ini_phi(name, ini_list, phi_list):
         rates = Parallel(n_jobs=-1)(delayed(get_rate)(name, ini, phi) for ini in ini_list for phi in phi_list)
         rates = np.array(rates)
         rates = rates.reshape(len(ini_list), len(phi_list), 32000, -1)
         return rates

    def get_df_ini_phi(rates):
        n_trials, n_phi, n_neurons, n_times = rates.shape
    
        # Create indices
        trials_ind, phi_ind, neurons_ind, times_ind = np.indices((n_trials, n_phi, n_neurons, n_times))
    
        # Construct DataFrame
        df = pd.DataFrame({
            'trial': trials_ind.flatten(),
            'phi': phi_ind.flatten(),
            'neuron': neurons_ind.flatten(),
            'time': times_ind.flatten(),
            'rates': rates.flatten()
        })
    
        return df

    def load_data_ini_phi(name, ini_list, phi_list):
        rates = get_rates_ini_phi(name, ini_list, phi_list)
        df = get_df_ini_phi(rates)
        return df

    def get_code(df):
        df_code = df.groupby(['time', 'trial', 'phi'])['rates'].apply(decode_bump).reset_index()
        df_code[['m0', 'm1', 'phase']] = pd.DataFrame(df_code['rates'].tolist(), index=df_code.index)
        df_code = df_code.drop(columns=['rates'])
    
        end_point = df_code[df_code.time==df_code.time.iloc[-1]]
        end_point = end_point.drop(columns=['time'])
        return df_code, end_point

    def get_precision(x):
    
        cmean =  (x - circmean(x, low=-np.pi, high=np.pi)) 
    
        cmean[cmean > np.pi] -= 2*np.pi
        cmean[cmean < -np.pi] += 2*np.pi
    
        return cmean



## Neuromodulation



### Parameters



    ini_list = np.arange(0, 20)
    phi_list = np.linspace(0, 360, 9)[1:]
    conf_name = 'config'
    name = 'last2'
    
    gain = 1.2
    ugain = 2.0
    dgain = 0.8
    # phi_list = [180]
    # ini_list = [0]

    1.95*1.15

    2.2424999999999997



### Simulations



#### off and on



    session = name + '_off'
    run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...

    session = name + '_on'
    run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, gain=gain)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



#### Jab and corr



##### Excitatory



    session = name + '_Jee'
    #  run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=1.2, X_axis=[0], gain=1.2)

    session = name + '_Jie'
    #  run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=1.2, X_axis=[2], gain=1.2)

    session = name + '_Jae'
    run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=ugain, X_axis=[0, 2], gain=gain)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



##### inhibitory



    session = name + '_Jei'
    #  run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=0.95, X_axis=[1], gain=1.2)

    session = name + 'odr_Jii'
    # run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=0.95, X_axis=[3], gain=1.2)

    session = name +'_Jai'
    run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=dgain, X_axis=[1, 3], gain=gain)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



##### other



    session = name + '_corr'
    run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='CORR_FF', X_gain=0.5, X_axis=[0, 1], gain=gain)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...

    session = name + '_tauf'
    #  run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='TAU_FAC', X_gain=1.1, X_axis=[0], gain=1.2)

    session = name + '_use'
    #  run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='USE', X_gain=1.1, X_axis=[0], gain=1.2)

    session = name + '_Jaa'
    # run_ini_phi(REPO_PATH, conf_name, session, ini_list, phi_list, X_str='Jab', X_gain=1.2, X_axis=[0, 2], gain=1.2)



### Data



#### Load



    def get_smooth(df_smooth):
      smooth = []
      for i in range(df_smooth.rates.shape[0]):
          m0, m1, phase = decode_bump(df_smooth.rates[i])
          smooth.append(np.roll(df_smooth.rates[i], int((phase / 2.0 / np.pi - 0.5) * df_smooth.rates[i].shape[0])))
    
      return np.array(smooth)

    def load_df(session):
      df = load_data_ini_phi(session, ini_list, phi_list)
      df_code, end_point = get_code(df)
    
      end_point['precision'] = end_point.groupby(['phi'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
      df_smooth = df.groupby(['time', 'trial', 'phi'])['rates'].apply(circcvl).reset_index()
    
      smooth = []
      for i in range(df_smooth.rates.shape[0]):
          m0, m1, phase = decode_bump(df_smooth.rates[i])
          smooth.append(np.roll(df_smooth.rates[i], int((phase / 2.0 / np.pi - 0.5) * df_smooth.rates[i].shape[0])))
    
      smooth = np.array(smooth)
    
      return df_code, end_point, smooth



#### Plots



    IF_LOAD = 1
    bins = 'auto'
    N_E = 32000
    titles = ['FF Input $\\nearrow$', '$J_{AE} \\nearrow$','$J_{AI} \\searrow$','FF Corr $\\searrow$']
    sessions = ['_on', '_Jae', '_Jai', '_corr']
    inset_titles = ['ns', '*', '*', '*']
    
    fig, ax = plt.subplots(2, 4, figsize=[3*width, 2*height], sharey='row', sharex='row')
    
    if IF_LOAD:
        df_code, end_point, smooth = load_df(name + '_off')
        end_point['syn'] = 0
    
        df_list= [df_code]
        smooth_list = [smooth]
        end_list = [end_point]
    
    ax[0][0].set_ylabel('Firing Rate (Hz)')
    ax[0][0].set_ylim([0, 15])
    
    ax[1][0].set_ylabel('Density')
    ax[1][0].set_ylim([0, 0.125])
    ax[0][0].set_title('%s' % titles[0])
    
    for i in range(len(sessions)):
    
        if IF_LOAD:
            session = name + sessions[i]
            df_code_on, end_point_on, smooth_on = load_df(session)
            end_point_on['syn'] = 1
    
            df_list.append(df_code_on)
            smooth_list.append(smooth_on)
            end_list.append(end_point_on)
    
        plot_smooth(smooth_list[0], ax[0][i], color=pal[0])
        plot_smooth(smooth_list[i+1], ax[0][i], color=pal[1])
    
        ax[0][i].set_xticks([0, N_E/2, N_E], [0, 180, 360])
        ax[0][i].set_xlabel('Pref. Location (°)')
        # ax[0][i].set_ylim([0, 15])
        if i>0:
            ax[0][i].set_title('%s, %s' % (titles[0], titles[i]))
    
        sns.histplot(data=end_list[0], x=end_list[0]['precision'], legend=False, ax=ax[1][i], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[0])
    
        sns.histplot(data=end_list[i+1], x=end_list[i+1]['precision'], legend=False, ax=ax[1][i], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[1])
    
        ax[1][i].set_xlabel('Endpoint Deviation (°)')
        ax[1][i].set_xlim([-20, 20])
    
        if i>0:
            ax[1][i].set_ylabel("")
    
        inset_ax = inset_axes(ax[1][i], width="35%", height="35%", loc='upper right')
        end = pd.concat((end_list[0], end_list[i+1]))
        # Plot your inset data
        sns.lineplot(end, x='syn', y=end['precision'].abs(), marker='o', ax=inset_ax, err_style='bars', ms=6.5, lw=0)
        inset_ax.set_title(inset_titles[i], fontsize=18)
        inset_ax.set_xlim([-1, 2])
        inset_ax.set_ylabel('')
    
        if i==2:
            inset_ax.set_ylim([2, 10])
            inset_ax.set_yticks([2, 10])
        else:
            inset_ax.set_ylim([2, 8])
            inset_ax.set_yticks([2, 8])
    
        inset_ax.yaxis.tick_right()
    
        if i==0:
            inset_ax.set_ylabel('Std (°)')
            inset_ax.yaxis.set_label_position('right')
    
        inset_ax.xaxis.set_visible(False)
        inset_ax.spines['bottom'].set_visible(False)
        inset_ax.spines['left'].set_visible(False)
        inset_ax.spines['right'].set_visible(True)
    
        if i>0:
            ax[0][i].yaxis.set_visible(False)
            ax[0][i].spines['left'].set_visible(False)
    
            ax[1][i].yaxis.set_visible(False)
            ax[1][i].spines['left'].set_visible(False)
    
    plt.savefig(name + '_neuro.svg', dpi=300)
    plt.show()

    /tmp/ipykernel_1407531/2581785449.py:88: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
      plt.savefig(name + '_neuro.svg', dpi=300)
    /home/leon/mambaforge/envs/dual_data/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
      fig.canvas.print_figure(bytes_io, **kw)

![img](./.ob-jupyter/43112e854f8eb77195271d553eec65c7fe4be5de.png)

    end = pd.concat(end_list)
    # print(end.head())

    fig, ax = plt.subplots(2, 1, figsize=(0.9 * width, 2*height))
    
    sns.lineplot(end, x='syn', y=end['m1']/end['m0'], marker='o', ax=ax[0], err_style='bars', ms=10, lw=0)
    ax[0].set_xlabel('NB Modulation')
    ax[0].set_ylabel('Amplitude, $\mathcal{F}_1 / \mathcal{F}_0$')
    ax[0].set_ylim([0.25, 1.25])  
    ax[0].set_xticks([0, 1, 2, 3, 4], ['OFF', 'ON', '$J_{AE} \\nearrow$', '$J_{AI} \\searrow$', '$CORR  \\searrow$'], fontsize=16)
    
    sns.lineplot(end, x='syn', y=end['precision'].abs(), marker='o', ax=ax[1], err_style='bars', ms=10, lw=0)
    ax[1].set_xticks([0, 1, 2, 3, 4], ['OFF', 'ON', '$J_{AE} \\nearrow$', '$J_{AI} \\searrow$', '$CORR  \\searrow$'], fontsize=16)
    ax[1].set_xlabel('NB Modulation')
    ax[1].set_ylabel('Response STD (°)')
    ax[1].set_ylim([2, 10])
    plt.savefig('apanel.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/a8a19629df7a6b6683e7f83278ed3dd42b49e183.png)

